In [1]:
import pandas as pd
import numpy as np
import pickle
import math
from time import time

from sklearn.preprocessing import QuantileTransformer, MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer, MaxAbsScaler, Normalizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.layers import Dense, LSTM
from keras.models import Sequential
from keras.callbacks.callbacks import Callback
from keras import optimizers

Using TensorFlow backend.


In [2]:
def getDf():
    df = pd.read_csv('train.csv')
    df = df.dropna()
    df = df.drop(columns=['resp_1','resp_2','resp_3','resp_4','ts_id'])
    
    return df

In [3]:
class CustomJumper(Callback):
    def on_epoch_end(self, epoch, logs={}):
        global t
        t = time() - t
        if(t > 2.16e4):
            error = True
            self.model.stop_training = True

In [4]:
def getTVT_split(split):
    df = getDf()
    y = np.where(df['resp'] > 0,1,0).reshape(len(df),1)
    x = df.values
    
    x_train, x_test = train_test_split(x,train_size=split,random_state=5)

    y_train, y_test = train_test_split(y,train_size=split,random_state=5)
    
    return x_train, x_test, y_train, y_test

In [5]:
def fitter(gen,memb):
    split = round(int(np.array2string(gen[:15],separator='')[1:-1],2)/10**5 + 0.5,5)
    hid_layers = 21
    act_in = act[int(int(np.array2string(gen[15:22],separator='')[1:-1],2)/11)]
    nn_in = int(np.array2string(gen[22:29],separator='')[1:-1],2) + 1
    act_out = act[int(int(np.array2string(gen[29:36],separator='')[1:-1],2)/11)]
    
    nn_hid = np.zeros(21).astype('int')
    for i,j in zip(range(36,183,7),range(21)):
        nn_hid[j] = int(np.array2string(gen[i:i+7],separator='')[1:-1],2)
        
    act_hid = np.zeros(21).astype('str')
    for i,j in zip(range(183,330,7),range(21)):
        act_hid[j] = act[int(int(np.array2string(gen[i:i+7],separator='')[1:-1],2)/11)]
           
    sc_each = list()
    for i,j in zip(range(330,850,4),range(130)):
        sc = int(np.array2string(gen[i:i+4],separator='')[1:-1],2)
        if sc == 0:
            scalerX = QuantileTransformer(output_distribution='normal',random_state=5)
        elif sc == 1:
            scalerX = QuantileTransformer(random_state=5)
        elif sc == 2:
            scalerX = MinMaxScaler()
        elif sc == 3:
            scalerX = StandardScaler()
        elif sc == 4:
            scalerX = StandardScaler(with_mean=True,with_std=False)
        elif sc == 5:
            scalerX = StandardScaler(with_mean=False,with_std=True)
        elif sc == 6:
            scalerX = StandardScaler(with_mean=False,with_std=False)
        elif sc == 7:
            scalerX = RobustScaler()
        elif sc == 8:
            scalerX = RobustScaler(with_centering=False,with_scaling=True)
        elif sc == 9:
            scalerX = RobustScaler(with_centering=True,with_scaling=False)
        elif sc == 10:
            scalerX = RobustScaler(with_centering=False,with_scaling=False)
        elif sc == 11:
            scalerX = PowerTransformer()
        elif sc == 12:
            scalerX = "no"
        elif sc == 13:
            scalerX = PowerTransformer(standardize=False)
        elif sc == 14:
            scalerX = MinMaxScaler(feature_range=(-1,1))
        elif sc == 15:
            scalerX = MaxAbsScaler()
        sc_each.append(scalerX)
        
    ls = losses[int(int(np.array2string(gen[850:857],separator='')[1:-1],2)/10)]
    opt = opts[int(np.array2string(gen[857:860],separator='')[1:-1],2)]
    
    if opt == 'adamT':
        opt = optimizers.Adam(amsgrad=True)
    
    print('Member '+str(memb)+': '+'Loading data...',end='\r',flush=True)
    x_train, x_test, y_train, y_test = getTVT_split(split)
    

    for i in range(130):
        print('Member '+str(memb)+': '+'Scaling '+str(i)+'...',end='\r',flush=True)
        if(sc_each[i]!="no"):
            x_train[:,i+3:i+4] = sc_each[i].fit_transform(x_train[:,i+3:i+4])
            x_test[:,i+3:i+4] = sc_each[i].transform(x_test[:,i+3:i+4])      
    
    model = Sequential()
    if act_in == '0':
        model.add(Dense(nn_in,input_dim=(130)))
    else:
        model.add(Dense(nn_in,input_dim=(130),activation=act_in))

    for i in range(21):
        if nn_hid[i] == 0:
            continue
        elif act_hid[i] == '0':
            model.add(Dense(nn_hid[i]))
        else:
            model.add(Dense(nn_hid[i],activation=act_hid[i]))

    if act_out == '0':
        model.add(Dense(y_train.shape[1]))
    else:
        model.add(Dense(y_train.shape[1],activation=act_out))

    model.compile(loss=ls,optimizer=opt)
    
    jumper = CustomJumper()
    eps = int(np.array2string(gen[860:865],separator='')[1:-1],2) + 1
    batch = int(np.array2string(gen[865:875],separator='')[1:-1],2) + 1
    print('Member '+str(memb)+': '+'Fitting model...',end='\r',flush=True)
    error = False
    t = time()
    model.fit(x_train[:,3:],y_train,epochs=eps,batch_size=batch,verbose=0,callbacks=[jumper])
    
    if(error):
        return 0
    
    print('Member '+str(memb)+': '+'Calculating LB...',end='\r',flush=True)
    results = model.predict_classes(x_test[:,3:])
    
    if ((np.count_nonzero(np.isnan(results)) > 0) or (np.count_nonzero(np.isinf(results)) > 0)):
        return 0
    
    LB_array = np.concatenate((x_test[:,:1],x_test[:,1:2]*x_test[:,2:3]*results),axis=1)
    K = (250/len(np.unique(LB_array[:,:1],axis=0)))**(1/2)
    LB_data = [np.sum(LB_array[:,1:][np.nonzero(LB_array[:,:1]==date)]) for date in np.unique(LB_array[:,:1])]
    t = np.sum(LB_data)/(np.sum(np.square(LB_data))**(1/2)) * K
    if(math.isnan(t)):
        t = 0
    LB1 = t*np.sum(LB_data)
    
    LB_array = np.concatenate((x_test[:,:1],x_test[:,1:2]*x_test[:,2:3]*y_test),axis=1)
    K = (250/len(np.unique(LB_array[:,:1],axis=0)))**(1/2)
    LB_data = [np.sum(LB_array[:,1:][np.nonzero(LB_array[:,:1]==date)]) for date in np.unique(LB_array[:,:1])]
    t = np.sum(LB_data)/(np.sum(np.square(LB_data))**(1/2)) * K
    if(math.isnan(t)):
        t = 0
    LB2 = t*np.sum(LB_data)
    
    if (LB2==0):
        return 0
    
    return LB1/LB2

In [6]:
act = ['0','linear','elu','selu','relu','sigmoid','hard_sigmoid','tanh','softmax','softplus','softsign','exponential']
losses = ['mean_squared_error','mean_absolute_error','mean_absolute_percentage_error','mean_squared_logarithmic_error','squared_hinge','hinge','categorical_hinge','logcosh','huber_loss','binary_crossentropy','kullback_leibler_divergence','poisson','cosine_proximity']
opts = ['sgd','rmsprop','adagrad','adadelta','adam','adamT','adamax','nadam']
n_gen = 20
adn = 875
t = 0

In [7]:
rd = np.random
rd.seed(5)

pop = list()
for i in range(20):
    gen = rd.randint(0,2,adn,'int')
    pop.append(gen)
pop = np.array(pop)
mez_pop = np.zeros(adn,'int').reshape(1,adn)
mez_fit = np.zeros(1,'int').reshape(1,1)

In [8]:
for j in range(0,n_gen+1):
    print('*****************************GENERATION '+str(j)+'*****************************')
    fit = list()
    for i in range(len(pop)):
        fit.append(fitter(pop[i],i))
    fit = np.array(fit)
    tot_fit = np.append(fit.reshape(20,1),mez_fit,axis=0)
    tot_pop = np.append(pop,mez_pop,axis=0)
    pickle.dump(tot_pop,open('Generations/pop_'+str(j)+'.pkl','wb'))
    pickle.dump(tot_fit,open('Generations/fit_'+str(j)+'.pkl','wb'))
    
    mez_pop = tot_pop[tot_fit[:,0].argsort()][-int(len(pop)/2):]
    mez_fit = tot_fit[tot_fit[:,0].argsort()][-int(len(pop)/2):]
    print('Best result:'+str(tot_fit[tot_fit[:,0].argsort()][-1:]))
    n_pop = list()
    for i in range(len(pop)):
        p1 = rd.randint(0,int(len(pop)/2))
        p2 = rd.randint(0,int(len(pop)/2))
        h = rd.randint(0,2,adn,'int')
        h = h.astype(bool)
        p1 = mez_pop[p1]
        p2 = mez_pop[p2]
        p1 = p1*h
        p2 = p2*np.invert(h)
        h = p1+p2

        mut = rd.rand(adn)
        mut = mut < 0.02
        h = (mut + h)%2

        n_pop.append(h)
    pop = np.array(n_pop)

*****************************GENERATION 0*****************************
Best result:[[0.00369857]]..
*****************************GENERATION 1*****************************
Best result:[[0.0037377]]...
*****************************GENERATION 2*****************************
Best result:[[0.00385486]]..
*****************************GENERATION 3*****************************
Best result:[[0.00406617]]..
*****************************GENERATION 4*****************************
Best result:[[0.00406617]]..
*****************************GENERATION 5*****************************


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\jonhe\anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3417, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-0f5435007cc7>", line 5, in <module>
    fit.append(fitter(pop[i],i))
  File "<ipython-input-5-fa42930019d1>", line 60, in fitter
    x_train, x_test, y_train, y_test = getTVT_split(split)
  File "<ipython-input-4-bea0f40a19f7>", line 2, in getTVT_split
    df = getDf()
  File "<ipython-input-2-23b9b0caebaa>", line 2, in getDf
    df = pd.read_csv('train.csv')
  File "C:\Users\jonhe\AppData\Roaming\Python\Python37\site-packages\pandas\io\parsers.py", line 685, in parser_f
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\jonhe\AppData\Roaming\Python\Python37\site-packages\pandas\io\parsers.py", line 463, in _read
    data = parser.read(nrows)
  File "C:\Users\jonhe\AppData\Roaming\Python\Python37\site-packages\pandas\io\parsers.py", line 1154, in read
    re

TypeError: object of type 'NoneType' has no len()